In [ ]:
import os
import dotenv

dotenv.load_dotenv()

POLYMARKET_TESTING_POLYSAFE_ADDRESS = os.getenv("POLYMARKET_TESTING_POLYSAFE_ADDRESS")
POLYMARKET_TESTING_AGENT_EOA = os.getenv("POLYMARKET_TESTING_AGENT_EOA")
POLYMARKET_TESTING_AGENT_EOA_PRIVATE_KEY = os.getenv("POLYMARKET_TESTING_AGENT_EOA_PRIVATE_KEY")  
USDCE_ADDRESS = os.getenv("USDCE_ADDRESS_POLYGON")
POLYGON_RPC = os.getenv("POLYGON_RPC_URL")  # Polygon rpc url

BUILDER_API_KEY = os.getenv("BUILDER_API_KEY")
BUILDER_SECRET = os.getenv("BUILDER_SECRET")
BUILDER_PASS_PHRASE = os.getenv("BUILDER_PASS_PHRASE")


In [ ]:
import os
from dotenv import load_dotenv
from eth_abi import encode
from py_builder_relayer_client.client import RelayClient
from py_builder_relayer_client.models import SafeTransaction, OperationType
from py_builder_signing_sdk.config import BuilderConfig, RemoteBuilderConfig

load_dotenv()

# -------------------------------------------------------------------
# CONFIG
# -------------------------------------------------------------------

CHAIN_ID = 137  # Polygon
RELAYER_URL = "https://relayer-v2.polymarket.com/"

POLYSAFE_ADDRESS = POLYMARKET_TESTING_POLYSAFE_ADDRESS
CONDITIONAL_TOKENS = "0x4D97DCd97eC945f40cF65F87097ACe5EA0476045"

COLLATERAL_TOKEN = USDCE_ADDRESS
PARENT_COLLECTION_ID = bytes.fromhex(
    "00" * 32
)

CONDITION_ID = bytes.fromhex(
    "249c21c68d86405e81fd7aabd5e34fa4035c66c11674d7159498ad1378a63062"
)

INDEX_SETS = [1]  # must be uint256[]

# -------------------------------------------------------------------
# ABI ENCODING
# -------------------------------------------------------------------

def encode_redeem_positions():
    selector = bytes.fromhex("01b7037c")  
    # keccak("redeemPositions(address,bytes32,bytes32,uint256[])")[:4]
    # FIXED: Was using wrong selector 9f0bb8a9

    encoded_args = encode(
        [
            "address",
            "bytes32",
            "bytes32",
            "uint256[]",
        ],
        [
            COLLATERAL_TOKEN,
            PARENT_COLLECTION_ID,
            CONDITION_ID,
            INDEX_SETS,
        ],
    )

    return selector + encoded_args


# -------------------------------------------------------------------
# MAIN
# -------------------------------------------------------------------

def main():
    # Configure remote builder (signing server)
    remote_builder_config = RemoteBuilderConfig(
        url="http://localhost:5001/sign",
    )
    builder_config = BuilderConfig(
        remote_builder_config=remote_builder_config,
    )
    
    # Create relay client with private key for signing
    client = RelayClient(
        relayer_url=RELAYER_URL,
        chain_id=CHAIN_ID,
        private_key=POLYMARKET_TESTING_AGENT_EOA_PRIVATE_KEY,
        builder_config=builder_config
    )

    calldata = encode_redeem_positions()

    # Create SafeTransaction object
    tx = SafeTransaction(
        to=CONDITIONAL_TOKENS,
        operation=OperationType.Call,
        data="0x" + calldata.hex(),
        value="0",
    )

    result = client.execute(
        transactions=[tx],
        metadata="Redeem conditional tokens"
    )

    print("Relayer response:")
    print(result)


if __name__ == "__main__":
    main()


In [ ]:
# Option 2: Using LOCAL builder config (with credentials from .env)
import os
from dotenv import load_dotenv
from eth_abi import encode
from py_builder_relayer_client.client import RelayClient
from py_builder_relayer_client.models import SafeTransaction, OperationType
from py_builder_signing_sdk.config import BuilderConfig, BuilderApiKeyCreds

load_dotenv()

# -------------------------------------------------------------------
# CONFIG
# -------------------------------------------------------------------

CHAIN_ID = 137  # Polygon
RELAYER_URL = "https://relayer-v2.polymarket.com/"

POLYSAFE_ADDRESS = POLYMARKET_TESTING_POLYSAFE_ADDRESS
CONDITIONAL_TOKENS = "0x4D97DCd97eC945f40cF65F87097ACe5EA0476045"

COLLATERAL_TOKEN = USDCE_ADDRESS
PARENT_COLLECTION_ID = bytes.fromhex(
    "00" * 32
)

CONDITION_ID = bytes.fromhex(
    "249c21c68d86405e81fd7aabd5e34fa4035c66c11674d7159498ad1378a63062"
)

INDEX_SETS = [2]  # must be uint256[]

# -------------------------------------------------------------------
# ABI ENCODING
# -------------------------------------------------------------------

def encode_redeem_positions():
    selector = bytes.fromhex("01b7037c")  
    # keccak("redeemPositions(address,bytes32,bytes32,uint256[])")[:4]
    # FIXED: Was using wrong selector 9f0bb8a9

    encoded_args = encode(
        [
            "address",
            "bytes32",
            "bytes32",
            "uint256[]",
        ],
        [
            COLLATERAL_TOKEN,
            PARENT_COLLECTION_ID,
            CONDITION_ID,
            INDEX_SETS,
        ],
    )

    return selector + encoded_args


# -------------------------------------------------------------------
# MAIN
# -------------------------------------------------------------------

def main():
    print("=" * 80)
    print("Redeem Positions Transaction (LOCAL Builder Config)")
    print("=" * 80)
    
    # Configure local builder with credentials from env
    print("\nConfiguring local builder with credentials from environment...")
    creds = BuilderApiKeyCreds(
        key=BUILDER_API_KEY,
        secret=BUILDER_SECRET,
        passphrase=BUILDER_PASS_PHRASE,
    )
    builder_config = BuilderConfig(
        local_builder_creds=creds,
    )
    print("✓ Local builder config created")
    
    # Create relay client with private key for signing
    print("\nCreating relay client...")
    client = RelayClient(
        relayer_url=RELAYER_URL,
        chain_id=CHAIN_ID,
        private_key=POLYMARKET_TESTING_AGENT_EOA_PRIVATE_KEY,
        builder_config=builder_config
    )
    print("✓ Relay client created")

    print("\nEncoding transaction data...")
    calldata = encode_redeem_positions()
    print(f"✓ Calldata: 0x{calldata.hex()[:50]}... ({len(calldata)} bytes)")

    # Create SafeTransaction object
    print("\nCreating SafeTransaction...")
    tx = SafeTransaction(
        to=CONDITIONAL_TOKENS,
        operation=OperationType.Call,
        data="0x" + calldata.hex(),
        value="0",
    )
    print(f"✓ Transaction to: {tx.to}")

    print("\nExecuting transaction via relayer...")
    result = client.execute(
        transactions=[tx],
        metadata="Redeem conditional tokens"
    )

    print("\n" + "=" * 80)
    print("SUCCESS!")
    print("=" * 80)
    print("\nTransaction Details:")
    print(f"  Transaction ID: {result.transaction_id}")
    print(f"  Transaction Hash: {result.transaction_hash}")


if __name__ == "__main__":
    main()


In [ ]:
# Check conditional token balances before redeeming
from web3 import Web3
from eth_abi import encode

w3 = Web3(Web3.HTTPProvider(POLYGON_RPC))

CONDITIONAL_TOKENS = "0x4D97DCd97eC945f40cF65F87097ACe5EA0476045"
COLLATERAL_TOKEN = USDCE_ADDRESS

PARENT_COLLECTION_ID = bytes.fromhex("00" * 32)
CONDITION_ID = bytes.fromhex(
    "0x276f0a3a9032afa889f8e1b1dd67278cf74ed1e050c79b9af705f946467ab646"
)

print("=" * 80)
print("CHECKING CONDITIONAL TOKEN BALANCES")
print("=" * 80)

# Calculate position IDs for the condition
print(f"\nPolySafe Address: {POLYSAFE_ADDRESS}")
print(f"Condition ID: {CONDITION_ID.hex()}")
print(f"Collateral Token: {COLLATERAL_TOKEN}")

# Calculate collection ID for index set [1]
# collectionId = keccak256(abi.encode(conditionId, indexSet))
index_set = 1  # This represents the index set as a bitmap
collection_id_input = encode(
    ["bytes32", "uint256"],
    [CONDITION_ID, index_set]
)
collection_id = w3.keccak(collection_id_input)
print(f"\nCollection ID (index set {index_set}): {collection_id.hex()}")

# Calculate position ID
# positionId = keccak256(abi.encode(collateralToken, collectionId))
position_id_input = encode(
    ["address", "bytes32"],
    [COLLATERAL_TOKEN, collection_id]
)
position_id = w3.keccak(position_id_input)
print(f"Position ID: {position_id.hex()}")

# Check balance using balanceOf(address,uint256)
balance_selector = w3.keccak(text="balanceOf(address,uint256)")[:4]
balance_calldata = balance_selector + encode(
    ["address", "uint256"],
    [POLYSAFE_ADDRESS, int.from_bytes(position_id, 'big')]
)

balance_result = w3.eth.call({
    'to': CONDITIONAL_TOKENS,
    'data': balance_calldata.hex()
})

balance = int.from_bytes(balance_result, 'big')
print(f"\n✓ Conditional Token Balance: {balance}")
print(f"  ({balance / 1e6} USDC.e equivalent)")

if balance == 0:
    print("\n⚠️  WARNING: Balance is 0! Cannot redeem.")
    print("   You need to have conditional tokens to redeem.")
else:
    print("\n✓ Balance > 0, redemption should be possible")
    
# Also check if the condition is resolved
print("\n" + "=" * 80)
print("CHECKING CONDITION STATUS")
print("=" * 80)

# Check payoutNumerators to see if condition is resolved
# getOutcomeSlotCount(bytes32)
outcome_count_selector = w3.keccak(text="getOutcomeSlotCount(bytes32)")[:4]
outcome_count_calldata = outcome_count_selector + CONDITION_ID

try:
    outcome_count_result = w3.eth.call({
        'to': CONDITIONAL_TOKENS,
        'data': outcome_count_calldata.hex()
    })
    outcome_count = int.from_bytes(outcome_count_result, 'big')
    print(f"Outcome Slot Count: {outcome_count}")
    
    # Check payoutNumerators for first outcome
    payout_selector = w3.keccak(text="payoutNumerators(bytes32,uint256)")[:4]
    payout_calldata = payout_selector + encode(
        ["bytes32", "uint256"],
        [CONDITION_ID, 0]
    )
    
    payout_result = w3.eth.call({
        'to': CONDITIONAL_TOKENS,
        'data': payout_calldata.hex()
    })
    payout = int.from_bytes(payout_result, 'big')
    
    if payout == 0:
        print(f"\n⚠️  WARNING: Condition NOT RESOLVED yet!")
        print("   Payout numerator is 0, which means the condition hasn't been resolved.")
        print("   You can only redeem after the market is resolved.")
    else:
        print(f"\n✓ Condition IS RESOLVED")
        print(f"  Payout numerator for outcome 0: {payout}")
        
except Exception as e:
    print(f"\n✗ Error checking condition status: {e}")


In [ ]:
print("=== ACTUAL HTTP HEADERS COMPARISON ===\n")

# Patch requests to capture actual headers
import requests
captured_headers = {}

original_request = requests.Session.request

def capture_request_headers(self, method, url, **kwargs):
    if "polymarket.com" in url:
        headers = kwargs.get('headers', {})
        caller = "LOCAL" if hasattr(self, '_local_marker') else "REMOTE"
        captured_headers[caller] = {k: v for k, v in headers.items() if 'POLY' in k or 'poly' in k.lower()}
        print(f"\n{caller} SIGNING -> Relayer Headers:")
        for key in sorted(captured_headers[caller].keys()):
            print(f"  {key}: {captured_headers[caller][key]}")
    return original_request(self, method, url, **kwargs)

requests.Session.request = capture_request_headers

print("--- Testing LOCAL ---")
local_client._session._local_marker = True
try:
    local_result = local_client.submit_transaction(tx, chain_id=CHAIN_ID)
    print(f"✓ LOCAL succeeded: {local_result}")
except Exception as e:
    print(f"✗ LOCAL failed: {e}")

print("\n--- Testing REMOTE ---")
try:
    remote_result = remote_client.submit_transaction(tx, chain_id=CHAIN_ID)
    print(f"✓ REMOTE succeeded: {remote_result}")
except Exception as e:
    print(f"✗ REMOTE failed: {e}")

requests.Session.request = original_request

print("\n\n=== HEADER KEY COMPARISON ===")
if 'LOCAL' in captured_headers and 'REMOTE' in captured_headers:
    local_keys = set(captured_headers['LOCAL'].keys())
    remote_keys = set(captured_headers['REMOTE'].keys())
    print(f"LOCAL keys: {sorted(local_keys)}")
    print(f"REMOTE keys: {sorted(remote_keys)}")
    print(f"Keys only in LOCAL: {sorted(local_keys - remote_keys)}")
    print(f"Keys only in REMOTE: {sorted(remote_keys - local_keys)}")
    print(f"Common keys: {sorted(local_keys & remote_keys)}")